In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-21"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
24601,2024-12-21,Brasil Nbb,11:00,Brasília,São Paulo,1.78,1.89,168.5,1.86,1.80,-2.5,2.05,1.65,hEF9B10B,0.561798,0.529101,0.537634,0.555556,0.090898,0.6,0.6,NaN,NaN,249.512,214.514959,0.859738,102.124,12.422644,0.121643,129.346,128.772,142.68,113.60,0.0,0.0,0.0,0.0,0.042388,0.023184,0.152888,1.07,0.214,3.644860,0.511816,0.5,-0.011816,0.03,0.006,148.333333,0.550106,0.6,0.049894
24602,2024-12-21,Brasil Nbb,17:00,Caxias do Sul,Pinheiros,2.41,1.47,147.5,1.84,1.82,3.5,1.98,1.70,nVwYBVBi,0.414938,0.680272,0.543478,0.549451,0.095210,0.4,0.6,NaN,NaN,282.174,109.382749,0.387643,180.070,69.173421,0.384147,174.418,243.738,311.60,177.10,1.0,0.0,0.0,0.0,0.342619,0.007728,0.107603,-3.61,-0.722,-1.952909,0.443466,0.2,-0.243466,4.83,0.966,0.486542,0.365695,0.6,0.234305
24603,2024-12-21,Brasil Nbb,18:00,Pato,Mogi,1.28,3.24,156.5,1.80,1.86,-8.5,1.92,1.75,lt9I9uVN,0.781250,0.308642,0.555556,0.537634,0.089892,1.0,0.2,NaN,NaN,312.890,150.596914,0.481309,204.130,64.353101,0.315255,187.362,292.774,146.20,287.00,0.0,0.0,0.0,0.0,0.613243,0.023184,0.065509,-0.33,-0.066,-4.242424,0.584354,0.5,-0.084354,-0.90,-0.180,-12.444444,0.239129,0.2,-0.039129
24604,2024-12-21,Brasil Nbb,19:00,União Corinthians,Corinthians Paulista,1.64,2.06,164.5,1.83,1.83,-3.5,1.98,1.70,festA9t4,0.609756,0.485437,0.546448,0.546448,0.095193,0.6,0.2,NaN,NaN,184.242,33.894733,0.183969,120.048,27.654286,0.230360,168.624,213.350,205.92,102.24,0.0,0.0,0.0,0.0,0.160532,0.000000,0.107603,-1.38,-0.276,-2.318841,0.517274,0.5,-0.017274,6.49,1.298,0.816641,0.491188,0.8,0.308812
24605,2024-12-21,Espanha Acb,14:00,Breogan,Basquet Girona,1.57,2.43,159.5,1.83,1.93,-4.5,1.99,1.81,vo8Y4zhC,0.636943,0.411523,0.546448,0.518135,0.048465,0.6,0.2,NaN,NaN,418.626,330.638119,0.789817,277.130,173.489737,0.626023,210.216,422.876,113.96,571.74,0.0,1.0,0.0,0.0,0.304056,0.037612,0.066989,0.15,0.030,19.000000,0.489201,0.3,-0.189201,1.10,0.220,6.500000,0.246862,0.1,-0.146862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24915,2024-12-21,Itália Série A1 Feminina,16:30,Alpo VillaFranca F,Battipaglia F,5.40,1.12,140.5,1.80,1.86,11.5,1.91,1.79,tKoiZYRq,0.185185,0.892857,0.555556,0.537634,0.078042,0.0,0.6,NaN,NaN,325.756,216.317619,0.664048,291.800,99.751763,0.341850,0.000,460.050,147.40,269.28,0.0,0.0,0.0,0.0,0.928349,0.023184,0.045866,0.00,0.000,inf,0.000000,0.0,0.000000,0.87,0.174,0.689655,0.000000,0.0,0.000000
24916,2024-12-21,Kosovo Superliga,13:00,Sigal Prishtina,Bashkimi,1.56,2.21,161.5,1.83,1.83,-4.5,1.99,1.70,OxtxjzK2,0.641026,0.452489,0.546448,0.546448,0.093514,0.8,0.4,NaN,NaN,136.504,20.265669,0.148462,103.108,10.864841,0.105373,127.432,111.442,136.68,105.78,0.0,0.0,0.0,0.0,0.243830,0.000000,0.111144,-0.88,-0.176,-3.181818,0.000000,0.0,0.000000,-3.44,-0.688,-1.758721,0.000000,0.0,0.000000
24917,2024-12-21,Macedônia Do Norte Prva Liga,13:00,Kozuv,Rabotnicki,1.69,2.09,158.5,1.83,1.83,-2.5,1.83,1.83,MN68R911,0.591716,0.478469,0.546448,0.546448,0.070185,0.6,0.6,NaN,NaN,173.848,112.448962,0.646823,180.088,116.003915,0.644151,167.828,172.204,110.08,85.32,0.0,0.0,0.0,0.0,0.149652,0.000000,0.000000,-2.40,-0.480,-1.437500,0.000000,0.0,0.000000,2.70,0.540,2.018519,0.000000,0.0,0.000000
24918,2024-12-21,Nova Zelândia Tauihi,23:00,Mid-North Whai F,Tokomanawa Queens F,1.42,2.56,162.5,

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Eua Nba,Orlando Magic,Miami Heat,209.5,1.93,0.9590,Over
1,21:30,Eua Nba,Atlanta Hawks,Memphis Grizzlies,240.5,1.93,0.9759,Over
2,22:00,Eua Nba,Chicago Bulls,Boston Celtics,239.5,1.80,0.5093,Over
3,22:30,Eua Nba,San Antonio Spurs,Portland Trail Blazers,227.5,1.93,0.9560,Over
4,14:30,Alemanha Bbl,Hamburg,Ludwigsburg,161.5,1.85,1.0000,Over
5,00:00,Eua Ncaa,Loyola Marymount,Southern,136.5,1.91,0.7500,Over
6,09:00,Rússia Liga Vtb United,Saratov,Nizhny Novgorod,157.5,1.85,0.9988,Over
7,22:00,Eua Nba,Milwaukee Bucks,Washington Wizards,223.5,1.84,0.9857,Over
8,22:30,Eua Nba,Dallas Mavericks,Los Angeles Clippers,220.5,1.84,0.8456,Over
9,23:00,Eua Nba,Phoenix Suns,Detroit Pistons,224.5,1.81,0.8444,Over
